In [ ]:
import ipynb
import pandas as pd
import numpy as np
import requests
import api_key
import json
from tqdm import tqdm
from geopy import distance


from ipynb.fs.full.mmtc_scrape import *

In [ ]:
#clean companies df: no cleaning needed
df_companies

In [ ]:
df_locations.count

In [ ]:
#clean locations df

#drop empty columns
df_locations = df_locations.drop(['EMAIL ADDRESS', 'PHONE'], axis=1, errors='ignore')

#modify company names to matche the df_companies dataframe
df_locations.replace('MüV', 'MuV', inplace=True)
df_locations.replace('Sunnyside*', 'Sunnyside', inplace=True)

#left join companies onto locations on company name
merged_df = df_locations.merge(right=df_companies, how='left', left_on='COMPANY', right_on='Name')

# clean merged_df
merged_df = merged_df.drop('Name', axis=1)

#provide a detailed request string for geoapify
merged_df['Full Address'] = merged_df['ADDRESS'] + ", " + merged_df['CITY'] + ', FL ' + merged_df['ZIP CODE']

#get only the FLUENT Dispensaries
fluent_dispensaries = merged_df.loc[merged_df['COMPANY'] == 'FLUENT']


fluent_dispensaries.head()
merged_df.count()

In [ ]:
# miami_df_test = merged_df.loc[merged_df['CITY'].str.contains("Miami")]
# miami_df_test.count()

In [ ]:
#summary analysis: count dispensaries by MMTC
merged_df['COMPANY'].value_counts()


In [ ]:
example_df = fluent_dispensaries.loc[fluent_dispensaries['ADDRESS'].str.contains('175 NW')]
example_df

In [ ]:
#input raw location dataset
input_ = merged_df

# #provide a detailed request string for geoapify
# input_['Full Address'] = input_['ADDRESS'] + ", " + input_['CITY'] + ', FL ' + input_['ZIP CODE']
# # input_ = input_[['ADDRESS']]


#clean input dataset
input_['Full Address'] = input_['Full Address'].str.replace('11245 SW 211 Street, Miami, FL 33189','11245 SW 211th St, Miami, FL  33189, United States')


#define output dataframe
output = pd.DataFrame(columns=['ADDRESS','Latitude', 'Longitude'])

#define failed list
failed_df = pd.DataFrame(columns=['ADDRESS'])
input_.head()

#limit the requests to the city of Miami Only
#miami_df = input_.loc[input_['CITY'].str.contains("Miami")]

In [ ]:
#geolocate all addresses (get coordinates)

#for every location address,
for row in input_.iterrows():
        
    #generate geoapify request url
    # location_id = row[1]['Location ID']
    #location_name = row[1]['BT Location Name']
    request_string = row[1]['Full Address']
    query_url = f"https://api.geoapify.com/v1/geocode/search?text={request_string}&format=json&apiKey={api_key.api_key}"
    
    try:
        #get returned data as json. This code hits the API
        geo_data = requests.get(query_url).json()

        #select desired fields to return to dataframe
        lat = geo_data['results'][0]['lat']
        lon = geo_data['results'][0]['lon']
        # county = geo_data['results'][0]['county']
        # popularity = geo_data['results'][0]['rank']['popularity']
        # importance = geo_data['results'][0]['rank']['importance']

        #append the data to the results dataframe
        current_frame = pd.DataFrame({'ADDRESS':[request_string],'Latitude':[lat], 'Longitude':[lon]})
        frames = [output, current_frame]
        output = pd.concat(frames)
        
        print('Success')
    
    except:
        current_frame = pd.DataFrame({'ADDRESS':[request_string]})
        failed_df = pd.concat([failed_df, current_frame])
        
        print('Failed')
    
#print the head of the dataframe to ensure that data was successfully loaded
output.head()

#print the failed dataframes
failed_df

In [ ]:
output.count()

merged_df_with_coordinates = input_.merge(right=output, right_on='ADDRESS', left_on='Full Address', how='left')

# clean columns
merged_df_with_coordinates = merged_df_with_coordinates.drop('ADDRESS_y', axis=1)
merged_df_with_coordinates['ADDRESS'] = merged_df_with_coordinates['ADDRESS_x']
merged_df_with_coordinates = merged_df_with_coordinates.drop('ADDRESS_x', axis=1)
merged_df_with_coordinates = merged_df_with_coordinates[['COMPANY', 'ADDRESS', 'CITY', 'ZIP CODE', 'COUNTY', 'Phone', 'Email',
       'Authorization Status', 'License Number', 'Full Address', 'Latitude',
       'Longitude']]

merged_df_with_coordinates.count()
merged_df_with_coordinates.to_csv('./locations_with_coordinates.csv')

#get the fluent dispensaries in miami
fluent_dispensaries = merged_df_with_coordinates.loc[merged_df_with_coordinates['COMPANY'] == 'FLUENT']
fluent_dispensaries.head()

# merged_df_with_coordinates.dtypes
#merged_df_with_coordinates.columns

In [ ]:
# shortcut

# use csv instead of api to save on credits

merged_df_with_coordinates.read_csv('./locations_with_coordinates.csv')

In [ ]:

#run through each fluent dispensary, caclulate the travel time / distance to every other dispensary

def calculate_distance_to_another_dispensary(from_waypoint, to_waypoint):

    #Test case coordinates
    # from_waypoint = [25.928299,-80.203588]
    # to_waypoint = [25.571142,-80.369250]

    #design request query
    url = f"https://api.geoapify.com/v1/routing?waypoints={from_waypoint[0]},{from_waypoint[1]}|{to_waypoint[0]},{to_waypoint[1]}&mode=drive&details=instruction_details&apiKey={api_key.api_key}"

    try:
        result_ = requests.get(url).json()
        distance_in_meters = result_['features'][0]['properties']['distance']
        print('Success')
        
        #convert distance from meters to miles
        distance_in_miles = distance_in_meters * 0.0006213712

        return(distance_in_miles)

    except:

        print(f'Faliure: {url}')



# calculate_distance_to_another_dispensary()




In [ ]:
def find_distance(from_waypoint, to_waypoint):

    

    return(distance.distance(from_waypoint, to_waypoint).miles)

In [ ]:
fluent_dispensaries

In [ ]:

#radius in miles
radius = 5
results_dict = {'ADDRESS':[], 'competitors':[], 'Latitude': [], 'Longitude': []}
competitor_dict = {'FLUENT Dispensary': [], 'Competitor List': []}

for fluent_dispensary in fluent_dispensaries.iterrows():

    number_of_competitors = 0
    
    for competitor in merged_df_with_coordinates.iterrows():

        fluent_coordinates = [fluent_dispensary[1]['Latitude'], fluent_dispensary[1]['Longitude']]
        competitor_coordinates = [competitor[1]['Latitude'], competitor[1]['Longitude']]

        distance_to_competitor = find_distance(fluent_coordinates, competitor_coordinates)

        if distance_to_competitor <= radius and distance_to_competitor != 0:
            number_of_competitors += 1

            competitor_dict['FLUENT Dispensary'].append(fluent_dispensary[1]['Full Address'])
            competitor_dict['Competitor List'].append(competitor[1]['Full Address'])
    
    results_dict['ADDRESS'].append(fluent_dispensary[1]['ADDRESS'])
    results_dict['competitors'].append(number_of_competitors)
    results_dict['Latitude'].append(fluent_dispensary[1]['Latitude'])
    results_dict['Longitude'].append(fluent_dispensary[1]['Longitude'])

results_dict


In [21]:
results = pd.DataFrame(results_dict)

# cleaning
results['ADDRESS'] = results['ADDRESS'].str.upper()
results['Latitude'] = results['Latitude'].astype(float).round(3)
results['Longitude'] = results['Longitude'].astype(float).round(3)
results['results_address'] = results['ADDRESS']

results_export_df = results[['results_address', 'competitors']]

# export
results_export_df.to_csv('./results.csv', index=False)

results_export_df

,results_address,competitors
0,971 CASSAT AVENUE,8
1,2385 S FERDON BOULEVARD,1
2,500 E CHASE STREET,8
3,763 ATLANTIC BOULEVARD,6
4,301 E NINE ROAD,7
5,2090 U.S. 98,0
6,3460 US HWY 441,3
7,622 S FEDERAL HIGHWAY,9
8,9637 US HWY 98,1
9,2620 S US HWY 1,5


In [27]:
competitor_dict_df = pd.DataFrame(competitor_dict)
competitor_dict_df.rename(columns={'FLUENT Dispensary': 'results_address', 'Competitor List': 'competitor_address'}, inplace=True)
competitor_dict_df.to_csv('./competitor_list.csv', index=False)

In [22]:
# import location masterlist
masterlist_df = pd.read_csv('./Retail Location Master List - For Competitor Counts.csv', index=False)
masterlist_df.head()

,location_id,location_name,region
0,30,Cape Coral,Western
1,26,Casselberry,Central
2,23,Clearwater,Western
3,37,Coral Gables,Southern
4,41,Coral Springs,Southern
